# Week 3 Cousera Capstone Data Science Course - Task1

#### Let's import some of the needed libraries

## -------------------------------------------------
## Scrape Wikipedia Page for Tonronto geodata and present it in a DataFrame
## -------------------------------------------------

In [2]:
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes 
import requests 
from bs4 import BeautifulSoup

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from sklearn.cluster import KMeans

In [3]:
from urllib.request import urlopen
resp = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#print(resp.read())   

soup = BeautifulSoup(resp.read(),'html.parser')
soup = soup.find("table", attrs={"class":"wikitable sortable"})
#print(soup.prettify())

#soup.table

In [4]:

datasets = []
for tr in soup.find_all("tr")[1:]:
    dataset = [td.get_text() for td in tr.find_all('td')]  
    # Remove the "\n" in the last element of the list
    dataset[-1] = dataset[-1][:].replace('\n', '')
    
    # when Borough has a value but neighborhood is "Not Assigned" 
    # then assign the Borough's value to Neighborhood
    if(dataset[2]=='Not assigned' and dataset[1]!='Not assigned'): 
        dataset[2] = dataset[1]  
    else:
        None

    # when Borough is "Not Assigned regardless of the neighborhood value"    
    # then skip that has Borough
    if(dataset[1]=='Not assigned'): 
        None 
    else:
        datasets.append(dataset)


datasets[0:5]

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights']]

In [5]:
# The first tr contains the column names(headers).
headings = [th.get_text() for th in soup.find("tr").find_all("th")]
headings

['Postcode', 'Borough', 'Neighbourhood\n']

In [6]:
# Remove the "\n" in the last field of column name
headings[-1] = headings[-1][:].replace('\n', '')
headings

['Postcode', 'Borough', 'Neighbourhood']

In [7]:
raw_df=pd.DataFrame(datasets, columns=['Postcode', 'Borough', 'Neighborhood'])
#raw_df.head()

In [8]:
# get the unique Postcodes/Borough
group_raw_df = raw_df.groupby(['Postcode', 'Borough'],as_index=False).count()
group_raw_df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,2
1,M1C,Scarborough,3
2,M1E,Scarborough,3
3,M1G,Scarborough,1
4,M1H,Scarborough,1


In [9]:
# Combine neigbourhoods within same Postalcose/Borough into a single tuple
list1=[]

for row in group_raw_df.itertuples():
    string1=''
    for nghbr in raw_df[raw_df.Postcode==row.Postcode].itertuples():
        
        if string1 =='': 
            string1 = (nghbr.Neighborhood)
        else :
            string1 = string1 +', '+(nghbr.Neighborhood)
    list1.append([row.Postcode,row.Borough,string1])
#print(list1)


new_df=pd.DataFrame(list1, columns=['Postcode', 'Borough', 'Neighborhood'])
new_df.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [10]:
new_df.shape

(103, 3)